In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import haversine as hs
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import csv

# სატრენინგო დატა ძალიან დიდია და მთლიანად არ ეტევა პანდას დატაფრეიმში. ამიტომ შეგვიძლია მხოლოდ პირველი 10_000_000 ჩანაწერი წამოვიღოთ.

In [2]:
N_ROWS = 10
train_df =  pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv', nrows = N_ROWS)
test_df =  pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')

# სანამ საქმეს შევუდგებოდეთ განვსაზღვროთ სხვადასხვა column-ების min-max მნიშვნელობები.

# fair_amount:

In [3]:
MIN_FAIR_AMOUNT = 2
MAX_FAIR_AMOUNT = 250

# distance:

In [4]:
MIN_DISTANCE = 0.5
MAX_DISTANCE = 150

# გავეცნოთ dataset-ის ზოგად სტატისტიკას, რათა აღმოვაჩინოთ უჩვეულო მნიშვნელობები.

In [5]:
train_df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,9.410000,-73.971771,40.741306,-73.968175,40.755059,1.200000
std,4.414232,0.048531,0.021147,0.046012,0.021317,0.421637
min,4.500000,-74.016048,40.711303,-73.993078,40.712278,1.000000
25%,6.150000,-73.997505,40.727942,-73.991466,40.750684,1.000000
50%,8.300000,-73.981698,40.733508,-73.984681,40.758115,1.000000
75%,11.325000,-73.971072,40.758868,-73.973119,40.763190,1.000000
max,16.900000,-73.844311,40.774138,-73.841610,40.783762,2.000000


# შევამოწმოთ გვაქვს თუ არა ე.წ. missing value-ები და ამის შესაბამისად ჩავატაროთ data cleaning.

In [6]:
print(train_df.isnull().sum())

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64


# missing value-ები გვაქვს, შესაბამისად გადავყაროთ ყველა ის სტრიქონი რომელიც missing value-ს შეიცავს.

In [7]:
train_df = train_df.dropna()
print('დავრწმუნდეთ რომ missing value-ები აღარ გვაქვს:')
print(train_df.isnull().sum())
print(f'გაფილტრულ მონაცემთა რაოდენობა: {N_ROWS - len(train_df)}')

დავრწმუნდეთ რომ missing value-ები აღარ გვაქვს:
key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64
გაფილტრულ მონაცემთა რაოდენობა: 0


# როგორც train_df.describe()-მა გვიჩვენა: dataset-ში არის უარყოფითი fare_amount მნიშვნელობები, რაც ალოგიკურია რადგან მგზავრობის ფასი არ შეიძლება იყოს უარყოფითი.

In [8]:
negative_fare_amount = train_df[train_df['fare_amount'] < 0]
print(f'უარყოფითი fare_amount-ის მქონე სტრიქონების რაოდენობა: {len(negative_fare_amount)}')
print(negative_fare_amount.head())

უარყოფითი fare_amount-ის მქონე სტრიქონების რაოდენობა: 0
Empty DataFrame
Columns: [key, fare_amount, pickup_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, passenger_count]
Index: []


# ასეთი მონაცემები, ე.წ. outlier-ები ამოვშალოთ dataset-დან.

In [9]:
train_df = train_df[train_df['fare_amount'] >= MIN_FAIR_AMOUNT]

# train_df.describe()-მა ასევე გვიჩვენა, რომ dataset-ში fair_amount-ის მაქსიმალური მნიშვნელობა ძალიან დიდ მნიშვნელობას იძლევა რაც ასევე არაა რეალისტური. შესაბამისად ამგვარი მონაცემებიც გავფილტროთ:

In [10]:
train_df = train_df[train_df['fare_amount'] <= MAX_FAIR_AMOUNT]

In [11]:
train_df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,9.410000,-73.971771,40.741306,-73.968175,40.755059,1.200000
std,4.414232,0.048531,0.021147,0.046012,0.021317,0.421637
min,4.500000,-74.016048,40.711303,-73.993078,40.712278,1.000000
25%,6.150000,-73.997505,40.727942,-73.991466,40.750684,1.000000
50%,8.300000,-73.981698,40.733508,-73.984681,40.758115,1.000000
75%,11.325000,-73.971072,40.758868,-73.973119,40.763190,1.000000
max,16.900000,-73.844311,40.774138,-73.841610,40.783762,2.000000


# longitude და latitude მნიშვნელობები ცდება შესაბამის რეინჯებს: [-180, 180] და [-90, 90]. ასეთი სტრიქონებიც ამოვშალოთ:

In [12]:
MIN_LONGITUDE, MAX_LONGITUDE, MIN_LATITUDE, MAX_LATITUDE = -180, 180, -90, 90

# train_df pickup
train_df = train_df[train_df['pickup_longitude'] >= MIN_LONGITUDE]
train_df = train_df[train_df['pickup_longitude'] <= MAX_LONGITUDE]
train_df = train_df[train_df['pickup_latitude'] >= MIN_LATITUDE]
train_df = train_df[train_df['pickup_latitude'] <= MAX_LATITUDE]

# train_df dropoff
train_df = train_df[train_df['dropoff_longitude'] >= MIN_LONGITUDE]
train_df = train_df[train_df['dropoff_longitude'] <= MAX_LONGITUDE]
train_df = train_df[train_df['dropoff_latitude'] >= MIN_LATITUDE]
train_df = train_df[train_df['dropoff_latitude'] <= MAX_LATITUDE]

# test_df pickup
test_df = test_df[test_df['pickup_longitude'] >= MIN_LONGITUDE]
test_df = test_df[test_df['pickup_longitude'] <= MAX_LONGITUDE]
test_df = test_df[test_df['pickup_latitude'] >= MIN_LATITUDE]
test_df = test_df[test_df['pickup_latitude'] <= MAX_LATITUDE]

# test_df dropoff
test_df = test_df[test_df['dropoff_longitude'] >= MIN_LONGITUDE]
test_df = test_df[test_df['dropoff_longitude'] <= MAX_LONGITUDE]
test_df = test_df[test_df['dropoff_latitude'] >= MIN_LATITUDE]
test_df = test_df[test_df['dropoff_latitude'] <= MAX_LATITUDE]

# დავთვალოთ დისტანცია მგზავრობის საწყის და საბოლოო წერტილებს შორის, და ჩავამატოთ შესაბამისი მონაცემები ჩვენს dataset-ში:

In [13]:
def calculate_distance(df_row):
    pickup = df_row[['pickup_latitude', 'pickup_longitude']].values
    dropoff = df_row[['dropoff_latitude', 'dropoff_longitude']].values
    distances = hs.haversine_vector(pickup, dropoff, hs.Unit.KILOMETERS)
    return distances

train_df['distance'] = calculate_distance(train_df)
test_df['distance'] = calculate_distance(test_df)

# ასევე fare_amount-ის მსგავსი პრინციპით გავფილტროთ distance column-ც:

In [14]:
train_df = train_df[train_df['distance'] >= MIN_DISTANCE]
train_df = train_df[train_df['distance'] <= MAX_DISTANCE]
test_df = test_df[test_df['distance'] >= MIN_DISTANCE]
test_df = test_df[test_df['distance'] <= MAX_DISTANCE]

# ასევე გავფილტროთ passenger_count column-ც:

In [15]:
MAX_PASSENGER_COUNT = 5
train_df = train_df[train_df['passenger_count'] >= 0]
train_df = train_df[train_df['passenger_count'] <= MAX_PASSENGER_COUNT]

In [16]:
print(f'გაფილტვრის შემდეგ მონაცემების რაოდენობა: {len(train_df)}')
train_df.describe()

გაფილტვრის შემდეგ მონაცემების რაოდენობა: 10


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,9.410000,-73.971771,40.741306,-73.968175,40.755059,1.200000,2.927024
std,4.414232,0.048531,0.021147,0.046012,0.021317,0.421637,2.218073
min,4.500000,-74.016048,40.711303,-73.993078,40.712278,1.000000,1.030765
25%,6.150000,-73.997505,40.727942,-73.991466,40.750684,1.000000,1.431098
50%,8.300000,-73.981698,40.733508,-73.984681,40.758115,1.000000,2.399217
75%,11.325000,-73.971072,40.758868,-73.973119,40.763190,1.000000,3.552841
max,16.900000,-73.844311,40.774138,-73.841610,40.783762,2.000000,8.450145


# გადავიყვანოთ pickup_datetime column-ები შესაბამის ფორმატში:

In [17]:
train_df['pickup_datetime'] = pd.to_datetime(train_df['pickup_datetime'], errors='coerce')
test_df['pickup_datetime'] = pd.to_datetime(test_df['pickup_datetime'], errors='coerce')

# უფრო დეტალურად ამოვიღოთ ისეთივე feature-ები, როგორებიცაა: წელი, თვე, დღე, საათი, კვირის დღე:

In [18]:
train_df['pickup_year'] = train_df['pickup_datetime'].dt.year
train_df['pickup_month'] = train_df['pickup_datetime'].dt.month
train_df['pickup_day'] = train_df['pickup_datetime'].dt.day
train_df['pickup_hour'] = train_df['pickup_datetime'].dt.hour
train_df['pickup_day_of_week'] = train_df['pickup_datetime'].dt.dayofweek

test_df['pickup_year'] = test_df['pickup_datetime'].dt.year
test_df['pickup_month'] = test_df['pickup_datetime'].dt.month
test_df['pickup_day'] = test_df['pickup_datetime'].dt.day
test_df['pickup_hour'] = test_df['pickup_datetime'].dt.hour
test_df['pickup_day_of_week'] = test_df['pickup_datetime'].dt.dayofweek

In [19]:
print(f'მიღებული train dataset: {train_df.describe()}')
print(f'მიღებული test dataset: {test_df.describe()}')

მიღებული train dataset:        fare_amount  pickup_longitude  pickup_latitude  dropoff_longitude  \
count    10.000000         10.000000        10.000000          10.000000   
mean      9.410000        -73.971771        40.741306         -73.968175   
std       4.414232          0.048531         0.021147           0.046012   
min       4.500000        -74.016048        40.711303         -73.993078   
25%       6.150000        -73.997505        40.727942         -73.991466   
50%       8.300000        -73.981698        40.733508         -73.984681   
75%      11.325000        -73.971072        40.758868         -73.973119   
max      16.900000        -73.844311        40.774138         -73.841610   

       dropoff_latitude  passenger_count   distance  pickup_year  \
count         10.000000        10.000000  10.000000    10.000000   
mean          40.755059         1.200000   2.927024  2010.800000   
std            0.021317         0.421637   2.218073     1.229273   
min           40.71

# დავიმახსოვროთ test_df submission.csv ფაილის გენერაციისთვის:

In [20]:
X_test = test_df.copy()

# დავტოვოთ მხოლოდ ისეთი სტრიქონები, რომლებიც ნიუ იორკის საზღვრებშია:

In [21]:
bounds = (-74.25559, -73.70001, 40.49612, 40.91553)  # (min_lon, max_lon, min_lat, max_lat)

def is_in_nyc(longitude, latitude):
    min_longitude, max_longitude, min_latitude, max_latitude = bounds
    return (longitude >= min_longitude) and (longitude <= max_longitude) and (latitude >= min_latitude) and (latitude <= max_latitude)

train_df = train_df[train_df.apply(lambda row: is_in_nyc(row['pickup_longitude'], row['pickup_latitude']) and is_in_nyc(row['dropoff_longitude'], row['dropoff_latitude']), axis=1)]
test_df = test_df[test_df.apply(lambda row: is_in_nyc(row['pickup_longitude'], row['pickup_latitude']) and is_in_nyc(row['dropoff_longitude'], row['dropoff_latitude']), axis=1)]

In [22]:
print(f'დარჩენილ მონაცემთა რაოდენობა {len(train_df)}')
print(train_df.describe())
print(test_df.describe())

დარჩენილ მონაცემთა რაოდენობა 10
       fare_amount  pickup_longitude  pickup_latitude  dropoff_longitude  \
count    10.000000         10.000000        10.000000          10.000000   
mean      9.410000        -73.971771        40.741306         -73.968175   
std       4.414232          0.048531         0.021147           0.046012   
min       4.500000        -74.016048        40.711303         -73.993078   
25%       6.150000        -73.997505        40.727942         -73.991466   
50%       8.300000        -73.981698        40.733508         -73.984681   
75%      11.325000        -73.971072        40.758868         -73.973119   
max      16.900000        -73.844311        40.774138         -73.841610   

       dropoff_latitude  passenger_count   distance  pickup_year  \
count         10.000000        10.000000  10.000000    10.000000   
mean          40.755059         1.200000   2.927024  2010.800000   
std            0.021317         0.421637   2.218073     1.229273   
min        

# განვსაზღვროთ features მასივი:

In [23]:
features = [
    'pickup_longitude', 
    'pickup_latitude', 
    'dropoff_longitude', 
    'dropoff_latitude', 
    'passenger_count', 
    'distance', 
    'pickup_year', 
    'pickup_month', 
    'pickup_day', 
    'pickup_hour', 
    'pickup_day_of_week'
    ]
X_test = test_df.copy()

# წრფივი რეგრესიის მოდელი:

In [24]:
def linear_regression():
    X = train_df[features]
    y = train_df['fare_amount']
    

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    
    y_valid_pred = model.predict(X_valid)
    rmse_valid = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
    
    print(f'rmse_train: {rmse_train}')
    print(f'rmse_valid: {rmse_valid}')
    
    return model
    
model = linear_regression()
# generate_submission(model, X_test)

rmse_train: 3.616711983044361e-13
rmse_valid: 9.304118487274973


# შევქმნათ submission.csv:

In [25]:
submission_df = pd.DataFrame({'key': X_test['key'], 'fare_amount': model.predict(X_test[features])})
submission_df.to_csv('submission.csv', index=False)

# XGBoost მოდელი:

In [26]:
def xgboost_regression():
    X = train_df[features]
    y = train_df['fare_amount']
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42)
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    
    y_valid_pred = model.predict(X_valid)
    rmse_valid = np.sqrt(mean_squared_error(y_valid, y_valid_pred))

    print(f'rmse_train: {rmse_train}')
    print(f'rmse_valid: {rmse_valid}')
    
    return model
    
model = xgboost_regression()

rmse_train: 0.0005927812346869125
rmse_valid: 3.517876937445658
